# Function Processing Module

Move standard_action, sig_match, and set_method into the module

## Convert set_method, sig_match, standard_action into a helper class

The class can be initialized with signature and action definitions to create different objects for different function groups. When the object is applied to a function or string, it returns a corresponding function with the expected argument signature.

The class object is a custom decorator function.

## Allow processing functions to take specific keyword arguments from context

add a wrapper to trap unwanted context items

1. Get parameters and defaults for all arguments of the
   supplied function.
2. Generate a defaults dictionary for all keyword
   arguments.
3. When called, pass non-keyword arguments directly to
   the original function.
4. Update the defaults dictionary with calling
   keywords except the context keyword, if present.
5. If context is among the default keyword arguments
   and is supplied when called, add context to the keyword dictionary; otherwise search the supplied context for any keys in the defaults dictionary and use them to update the defaults dictionary.
10. If a **kwargs is present in the original function,
    pass all remaining context items using **context.



https://stackoverflow.com/questions/76304643/reuse-of-function-signature-type-hints-and-intellisense-args-kwargs
